Importing Necessary Librabries

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical


#necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

Loading the Dataset

In [2]:
data = pd.read_csv('/content/tweet_emotions.csv')

EDA

In [3]:
#listing first few values in the dataset
pd.set_option('display.max_colwidth',None)
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin on your call...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


In [4]:
#Detailed info of the dataset and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


#Data Preprocessing

In [5]:
#dropping unnecessary columns
data = data.drop(columns=['tweet_id'])

In [6]:
#Creating a Function for text preprocessing
def clean_text(text):
    # Lowercase text
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
     # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
     # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove numbers
    text = re.sub(r'\d', '', text)
    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a single string
    text = ' '.join(tokens)

    return text

# Apply text cleaning
data['cleaned_text'] = data['content'].apply(clean_text)

#Feature Extraction

In [8]:
#converting the text data into numerical features with TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Transforming text data into TF-IDF features
X = vectorizer.fit_transform(data['cleaned_text'])

#Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['sentiment'])

In [9]:
# Spliting the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Testing different machine learning models


#1.Traditional Machine Learning Models


In [10]:
# Logistic Regression model

# Defining the hyperparameter grid for Random Search
lr_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'max_iter': [100, 200, 300, 400, 500]
}
lr = LogisticRegression(max_iter=1000)
lr_random_search = RandomizedSearchCV(estimator=lr, param_distributions=lr_param_grid, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
#fitting the model
lr_random_search.fit(X_train, y_train)
best_lr = lr_random_search.best_estimator_
#Predicting
lr_preds = best_lr.predict(X_test)
print("Logistic Regression Best Parameters:", lr_random_search.best_params_)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))
print(classification_report(y_test, lr_preds, target_names=label_encoder.classes_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error

Logistic Regression Best Parameters: {'penalty': 'l2', 'max_iter': 500, 'C': 1}
Logistic Regression Accuracy: 0.346125
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.33      0.01      0.01       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.12      0.01      0.03       338
   happiness       0.34      0.35      0.34      1028
        hate       0.53      0.16      0.25       268
        love       0.51      0.39      0.44       762
     neutral       0.34      0.57      0.42      1740
      relief       0.37      0.02      0.04       352
     sadness       0.34      0.25      0.29      1046
    surprise       0.31      0.05      0.09       425
       worry       0.33      0.48      0.39      1666

    accuracy                           0.35      8000
   macro avg       0.27      0.18      0.18      8000
weighted avg   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Support Vector Machine
svm_model = SVC()
#fitting the model
svm_model.fit(X_train, y_train)
#Predicting
svm_preds = svm_model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))
print(classification_report(y_test, svm_preds, target_names=label_encoder.classes_))

SVM Accuracy: 0.34425
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       1.00      0.01      0.02       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.06      0.00      0.01       338
   happiness       0.34      0.33      0.34      1028
        hate       0.48      0.18      0.26       268
        love       0.51      0.38      0.44       762
     neutral       0.32      0.60      0.42      1740
      relief       0.50      0.02      0.03       352
     sadness       0.39      0.18      0.25      1046
    surprise       0.40      0.03      0.05       425
       worry       0.32      0.49      0.39      1666

    accuracy                           0.34      8000
   macro avg       0.33      0.17      0.17      8000
weighted avg       0.36      0.34      0.30      8000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Random Forest Classifier
rf = RandomForestClassifier()

# Defining the hyperparameter grid for Random Search
rf_param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_random_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_grid, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
#fitting the model
rf_random_search.fit(X_train, y_train)
best_rf = rf_random_search.best_estimator_
#Predicting
rf_preds = best_rf.predict(X_test)
print("Random Forest Best Parameters:", rf_random_search.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print(classification_report(y_test, rf_preds, target_names=label_encoder.classes_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Random Forest Best Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50}
Random Forest Accuracy: 0.33175
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.00      0.00      0.00       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.00      0.00      0.00       338
   happiness       0.34      0.23      0.28      1028
        hate       0.51      0.13      0.21       268
        love       0.49      0.35      0.41       762
     neutral       0.30      0.72      0.43      1740
      relief       0.00      0.00      0.00       352
     sadness       0.47      0.12      0.19      1046
    surprise       0.00      0.00      0.00       425
       worry       0.32      0.44      0.37      1666

    accuracy                           0.33      8000
   macro avg   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# Naive Bayes
nb = MultinomialNB()
# Defining the hyperparameter grid for Random Search
nb_param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}


nb_random_search = RandomizedSearchCV(estimator=nb, param_distributions=nb_param_grid, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
#fitting the model
nb_random_search.fit(X_train, y_train)
best_nb = nb_random_search.best_estimator_
nb_preds = best_nb.predict(X_test)
print("Naive Bayes Best Parameters:", nb_random_search.best_params_)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_preds))
print(classification_report(y_test, nb_preds, target_names=label_encoder.classes_))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Naive Bayes Best Parameters: {'alpha': 1}
Naive Bayes Accuracy: 0.318625
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.00      0.00      0.00       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.00      0.00      0.00       338
   happiness       0.35      0.27      0.31      1028
        hate       0.00      0.00      0.00       268
        love       0.52      0.28      0.36       762
     neutral       0.30      0.55      0.39      1740
      relief       0.00      0.00      0.00       352
     sadness       0.36      0.11      0.17      1046
    surprise       0.50      0.00      0.00       425
       worry       0.30      0.59      0.39      1666

    accuracy                           0.32      8000
   macro avg       0.18      0.14      0.13      8000
weighted avg       0.30      0.32      0.26      8000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#2.Deep Learning Models:


simple deep learning model using an LSTM network

In [14]:
# Tokenizing the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['cleaned_text'])
X_seq = tokenizer.texts_to_sequences(data['cleaned_text'])
X_pad = pad_sequences(X_seq, maxlen=100)

In [15]:
# One-hot encoding labels
y_cat = to_categorical(y)

In [16]:
#Spliting
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_pad, y_cat, test_size=0.2, random_state=42)

In [17]:
# Building the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

#Compiling model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
# Training the model
history = model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
450/450 [==============================] - 101s 215ms/step - loss: 2.0237 - accuracy: 0.3047 - val_loss: 1.9378 - val_accuracy: 0.3344
Epoch 2/5
450/450 [==============================] - 90s 200ms/step - loss: 1.8244 - accuracy: 0.3857 - val_loss: 1.9244 - val_accuracy: 0.3441
Epoch 3/5
450/450 [==============================] - 93s 207ms/step - loss: 1.7070 - accuracy: 0.4297 - val_loss: 1.9479 - val_accuracy: 0.3344
Epoch 4/5
450/450 [==============================] - 128s 286ms/step - loss: 1.5956 - accuracy: 0.4617 - val_loss: 2.0145 - val_accuracy: 0.3250
Epoch 5/5
450/450 [==============================] - 92s 205ms/step - loss: 1.4958 - accuracy: 0.4963 - val_loss: 2.0866 - val_accuracy: 0.3206


In [19]:
# Evaluating model
loss, accuracy = model.evaluate(X_test_dl, y_test_dl)
print(f'LSTM Model Accuracy: {accuracy}')

# Prediction
y_pred_dl = model.predict(X_test_dl)
y_pred_labels = y_pred_dl.argmax(axis=-1)
print(classification_report(y_test, y_pred_labels, target_names=label_encoder.classes_))

250/250 [==============================] - 12s 47ms/step - loss: 2.0806 - accuracy: 0.3224
LSTM Model Accuracy: 0.32237499952316284
250/250 [==============================] - 9s 35ms/step
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        19
     boredom       0.00      0.00      0.00        31
       empty       0.00      0.00      0.00       162
  enthusiasm       0.00      0.00      0.00       163
         fun       0.10      0.09      0.09       338
   happiness       0.30      0.36      0.33      1028
        hate       0.28      0.14      0.19       268
        love       0.40      0.41      0.40       762
     neutral       0.36      0.48      0.41      1740
      relief       0.28      0.05      0.09       352
     sadness       0.29      0.27      0.28      1046
    surprise       0.15      0.06      0.08       425
       worry       0.33      0.40      0.36      1666

    accuracy                           0.32      8000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation

In [31]:
# accuracy of all models

print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_preds))
print(f'LSTM Model Accuracy: {accuracy}')


Logistic Regression Accuracy: 0.346125
SVM Accuracy: 0.34425
Random Forest Accuracy: 0.33175
Naive Bayes Accuracy: 0.318625
LSTM Model Accuracy: 0.3018749952316284


In [36]:
#  test user input text with best model

# User input
user_input = input("Enter your text: ")
cleaned_text = clean_text(user_input)
user_input_vector = vectorizer.transform([cleaned_text])
predicted_sentiment = best_lr.predict(user_input_vector)[0]

# Print the predicted sentiment
print(f"Predicted sentiment: {label_encoder.classes_[predicted_sentiment]}")


Enter your text: What a beautiful day!!
Predicted sentiment: happiness


from above analysis we can understand that Logistic Regression gave the best accuracy for our model for emotion detection from Twitter Data